# Running ESM fold to test

We can run esmfold from hugging face Here

In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

import torch
import pandas as pd


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


Google drive

In [3]:
from google.colab import drive
drive.mount("/content/drive")
#drive path string to root
drive_root = '/content/drive/MyDrive'

Mounted at /content/drive


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device =  torch.device("cpu")
# Print the device being used
print(f'Using device: {device}')

In [ ]:
# Load model directly
from transformers import AutoTokenizer, EsmForProteinFolding
from accelerate.test_utils.testing import get_backend

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1",
    device_map="auto",
    offload_folder="/content/drive/MyDrive/Colabtmp") #support offloading

print("current device", device)
model = model.to(device)
# Additional optimizations:
model.esm = model.esm.half() # Uncomment to switch the stem to float16
torch.backends.cuda.matmul.allow_tf32 = True
model.trunk.set_chunk_size(64)

model.eval()

Encoding part:

In [ ]:

# Example protein sequence
test_protein = "MGAGASAEEKHSRELEKKLKEDAEKDARTVKLLLLGAGESGKSTIVKQMKIIHQDGYSLEECLEFIAIIYGNTLQSILAIVRAMTTLNIQYGDSARQDDARKLMHMADTIEEGTMPKEMSDIIQRLWKDSGIQACFERASEYQLNDSAGYYLSDLERLVTPGYVPTEQDVLRSRVKTTGIIETQFSFKDLNFRMFDVGGQRSERKKWIHCFEGVTCIIFIAALSAYDMVLVEDDEVNRMHESLHLFNSICNHRYFATTSIVLFLNKKDVFFEKIKKAHLSICFPDYDGPNTYEDAGNYIKVQFLELNMRRDVKEIYSHMTCATDTQNVKFVFDAVTDIIIKENLKDCGLF"

tokenized_input = tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)['input_ids']
tokenized_input = tokenized_input.to(device)

In [ ]:
print(tokenized_input)

In [ ]:
# Run inference
with torch.no_grad():
    output = model(tokenized_input)

PDB conversion part and saving files

In [ ]:
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37

def convert_outputs_to_pdb(outputs):
    final_atom_positions = atom14_to_atom37(outputs["positions"][-1], outputs)
    outputs = {k: v.to("cpu").numpy() for k, v in outputs.items()}
    final_atom_positions = final_atom_positions.cpu().numpy()
    final_atom_mask = outputs["atom37_atom_exists"]
    pdbs = []
    for i in range(outputs["aatype"].shape[0]):
        aa = outputs["aatype"][i]
        pred_pos = final_atom_positions[i]
        mask = final_atom_mask[i]
        resid = outputs["residue_index"][i] + 1
        pred = OFProtein(
            aatype=aa,
            atom_positions=pred_pos,
            atom_mask=mask,
            residue_index=resid,
            b_factors=outputs["plddt"][i],
            chain_index=outputs["chain_index"][i] if "chain_index" in outputs else None,
        )
        pdbs.append(to_pdb(pred))
    return pdbs

In [ ]:
pdb = convert_outputs_to_pdb(output)

In [ ]:
print(pdb)

In [ ]:
#save to a pdb file

In [ ]:
file_name = "output.pdb"
path = f"{drive_root}/Colab_Data/{file_name}"
with open(path, "w") as f:
    f.writelines(pdb)
#delete explicitly to free up memory
del output
del pdb

# Running on all test sequences

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/CASP_dataset.csv")
print(len(data))
data.head(n=1)

In [ ]:
for index, row in data.iterrows():
  if index < 52:
    continue
  # Example protein sequence
  test_protein = row["sequence"]
  test_id = row["Target"]
  print("current index:", index, "| id: ",test_id )
  #see if sequence length is limited
  if len(test_protein) > 1022:
    test_protein = test_protein[:1022]
    print("sequence truncated")

  #limit to the given limit
  test_protein = str(test_protein[:1022])
  tokenized_input = tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)['input_ids']
  tokenized_input = tokenized_input.to(device)
  # Run inference
  with torch.no_grad():
      output = model(tokenized_input)
  pdb = convert_outputs_to_pdb(output)
  file_name = f"{test_id}.pdb"
  path = f"{drive_root}/Colab_Data/{file_name}"
  with open(path, "w") as f:
      f.writelines(pdb)
  #delete explicitly to free up memory
  del output
  del pdb

In [ ]:
# Run inference
with torch.no_grad():
    output = model(tokenized_input)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')